# Deploy code as an MLRun function

In [71]:
from mlrun import code_to_function, mlconf,mount_v3io

In [72]:
fn = code_to_function(project='Iguazio-Training',name='save_properities', kind='job',filename='modules/mlrun_functions.ipynb')

In [73]:
fn.spec.image = "mlrun/mlrun"

In [74]:
fn.apply(mount_v3io())

### Run interactive

In [75]:
params = {"source_file" : "/User/igztraining/mlrun/Day-4/data/demo.csv",
"target_file" : "/User/igztraining/mlrun/Day-4/data/condos.csv" }

In [76]:
save_properties = fn.run(name='run_withcode',artifact_path="/User/artifacts",handler='save_property',params=params,project='Iguazio-Training')

> 2020-09-01 13:55:19,195 [info] starting run run_withcode uid=7ca4bfc37fbf4f99aa736c4e21eed18e  -> http://mlrun-api:8080
> 2020-09-01 13:55:19,353 [info] Job is running in the background, pod: run-withcode-zx2vc
/User/igztraining/mlrun/Day-4/data/demo.csv
> 2020-09-01 13:55:23,795 [info] run executed, status=completed
final state: succeeded


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
Iguazio-Training,...21eed18e,0,Sep 01 13:55:23,completed,run_withcode,v3io_user=marcelokind=jobowner=marcelohost=run-withcode-zx2vc,,source_file=/User/igztraining/mlrun/Day-4/data/demo.csvtarget_file=/User/igztraining/mlrun/Day-4/data/condos.csv,properties_available=54,condo_properties


to track results use .show() or .logs() or in CLI: 
!mlrun get run 7ca4bfc37fbf4f99aa736c4e21eed18e --project Iguazio-Training , !mlrun logs 7ca4bfc37fbf4f99aa736c4e21eed18e --project Iguazio-Training
> 2020-09-01 13:55:25,467 [info] run executed, status=completed


In [77]:
save_properties.outputs

{'properties_available': 54,
 'condo_properties': 'store://Iguazio-Training/run_withcode_condo_properties#7ca4bfc37fbf4f99aa736c4e21eed18e'}

### Run step 2 with Step 1's output

In [78]:
params['city']="SACRAMENTO"
params['target_dataset']="sacramento_condos"

In [79]:
sp_run = fn.run(name='run_withcode',artifact_path="/User/artifacts",handler='filter_properties',
                inputs={"source_data":save_properties.outputs["condo_properties"]},
                params=params,project='Iguazio-Training')

> 2020-09-01 13:55:25,489 [info] starting run run_withcode uid=e4603f7554b744e69adc81c75d4d6553  -> http://mlrun-api:8080
> 2020-09-01 13:55:25,638 [info] Job is running in the background, pod: run-withcode-s92bp
> 2020-09-01 13:55:30,170 [info] run executed, status=completed
final state: succeeded


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
Iguazio-Training,...5d4d6553,0,Sep 01 13:55:29,completed,run_withcode,v3io_user=marcelokind=jobowner=marcelohost=run-withcode-s92bp,source_data,source_file=/User/igztraining/mlrun/Day-4/data/demo.csvtarget_file=/User/igztraining/mlrun/Day-4/data/condos.csvcity=SACRAMENTOtarget_dataset=sacramento_condos,,sacramento_condos


to track results use .show() or .logs() or in CLI: 
!mlrun get run e4603f7554b744e69adc81c75d4d6553 --project Iguazio-Training , !mlrun logs e4603f7554b744e69adc81c75d4d6553 --project Iguazio-Training
> 2020-09-01 13:55:31,790 [info] run executed, status=completed


# Run function as part of a pipeline

In [5]:
import kfp
from kfp import dsl
from mlrun import run_pipeline

In [6]:
@dsl.pipeline(
    name = 'Property Management',
    description = 'Condo reviews'
)
def job_pipeline(
source_file :str = ''
) -> None:
    """Define our pipeline.
    :param p1: A model parameter.
    """
    save_properties = fn.as_step(handler='save_property',
                            params={"source_file" : source_file,
"target_file" : "/User/igztraining/mlrun/data/condos.csv" },outputs=['condo_properties'])
    filter_properties = fn.as_step(handler='filter_properties',
                                   params={"source_file" : source_file,
"target_file" : "/User/igztraining/mlrun/data/condos.csv" },
                            inputs={"condo_properties":save_properties.outputs["condo_properties"]})

In [7]:
kfp.compiler.Compiler().compile(job_pipeline, 'jobpipe.yaml')
artifact_path = 'v3io:///users/admin/kfp/{{workflow.uid}}/'
arguments = {"source_file" : "/User/igztraining/mlrun/data/demo.csv"}
run_id = run_pipeline(job_pipeline, arguments, experiment='property_management', artifact_path=artifact_path)

> 2020-09-01 13:26:23,115 [info] using in-cluster config.


> 2020-09-01 13:26:23,511 [info] Pipeline run id=d4a76dbc-8f73-461c-94c1-a46f308339b0, check UI or DB for progress
